In [ ]:
import re
import string
import time
import requests
import regex as re
from bs4 import BeautifulSoup
import csv
import os
from os import path
import pandas as pd
from urllib.parse import urljoin

In [ ]:
# URL do site
url = "https://g1.globo.com/fato-ou-fake/"

# Fazendo a requisição HTTP
response = requests.get(url)

In [ ]:
# Função para buscar o conteúdo de um link
def fetch_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        print(f"Erro ao acessar {url}. Código de status: {response.status_code}")
        return None

In [ ]:
def extract_links_from_page(page_url):
    response = requests.get(page_url)
    # Faz a verificação se a conexão com o site foi bem sucedida e procura a tag <a> que possua em seu texto a marcação "É #Fake, que é um padrão usado para delimitar fake news"
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        articles_full_section = soup.find('div', class_="_evt")
        if articles_full_section:
            links = articles_full_section.find_all('a', string=lambda text: text and "É #FAKE" in text)
            return [link.get('href') for link in links]
    else:
        print(f"Erro ao acessar {page_url}. Código de status: {response.status_code}")
        return []

In [ ]:
def save_to_csv(data, filename='fato_ou_fake-Fake.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['titulo', 'textos', 'check'])
        for item in data:
            writer.writerow(item)

In [ ]:
# limpeza
def cleanSentence(phrase):
    # Convertendo para minúsculas
    phrase = phrase.lower()

    # Removendo URLs
    phrase = re.sub(r'http\S+|www\S+|https\S+', '', phrase, flags=re.MULTILINE)

    # Removendo números
    phrase = re.sub(r'\d+', '', phrase)

    # Removendo pontuação
    phrase = phrase.translate(str.maketrans('', '', string.punctuation))

    # Removendo espaços extras
    phrase = phrase.strip()

    # Removendo múltiplos espaços
    phrase = re.sub(r'\s+', ' ', phrase)

    return phrase


In [ ]:
def scrape_fato_ou_fake():
    base_url = "https://g1.globo.com/fato-ou-fake/index/feed/pagina-{}.ghtml"
    all_data = []
    processed_links = set()  # Conjunto para armazenar links já processados


    for page_num in range(1, 250):  # Itera pelas páginas 1 a 249
        page_url = base_url.format(page_num)
        print(f"Acessando página: {page_url}")
        links = extract_links_from_page(page_url)

        for link_url in links:
            # Ignora links vazios
            if not link_url:
                print("Link vazio encontrado. Ignorando...")
                continue

            # Ignora links duplicados
            if link_url in processed_links:
                print(f"Link duplicado encontrado: {link_url}. Ignorando...")
                continue

            # Adiciona o link ao conjunto de links processados
            processed_links.add(link_url)

            print(f"\nAcessando o link: {link_url}")
            content = fetch_content(link_url)
            if content:
                link_soup = BeautifulSoup(content, 'html.parser')
                title = link_soup.find('h1', class_='content-head__title')
                title_text = title.text.strip() if title else "Título não encontrado"

                # Extrai o texto em itálico
                italics = link_soup.find_all('em')
                italics_text = "\n".join([italic.text.strip() for italic in italics]) if italics else "Nenhum texto em itálico encontrado"

                # Limpa o título e o texto em itálico
                cleaned_title = cleanSentence(title_text)
                cleaned_italics_text = cleanSentence(italics_text)

                # Adiciona os dados à lista com o índice, título limpo, texto em itálico limpo e a flag "check" = "fake"
                all_data.append([cleaned_title, cleaned_italics_text, 'fake'])

    save_to_csv(all_data)
    print("Dados salvos em 'fato_ou_fake.csv'")

In [ ]:
if __name__ == "__main__":
    scrape_fato_ou_fake()

Acessando página: https://g1.globo.com/fato-ou-fake/index/feed/pagina-1.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/video/e-fake-video-de-paolla-oliveira-anunciando-desconto-de-50-em-camarote-no-carnaval-2025-13354819.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/19/e-fake-que-video-mostre-criminosos-deixando-delegacia-apos-recuperar-armas-de-grosso-calibre.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/18/e-fake-que-mamografia-aumenta-o-risco-de-cancer-de-mama.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/17/e-fake-video-em-que-famosos-usam-camiseta-com-estrela-de-davi-e-dedo-do-meio-para-kanye-west.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/video/e-fake-que-vinagre-de-alcool-e-recomendado-para-combater-mosquito-da-dengue-13347277.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/14/e-fake-foto-que-mostra-alexandre-de-moraes-usando-bone-da-usaid